In [15]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
import lightgbm as lgb
from lightgbm import LGBMRegressor
#from statsmodels.graphics.tsaplots import plot_acf
from datetime import date
import seaborn as sns
import string
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from catboost import CatBoostRegressor
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [16]:
beer_train = pd.read_csv("train_preprocessed.csv",dtype={'Store': object,'ts_promo': object,'store_promo': object,'dom_promo': object,'dow_promo': object})
beer_train['Date'] = pd.to_datetime(beer_train['Date'])

In [17]:
beer_test = pd.read_csv("test_preprocessed.csv",dtype={'Store': object,'ts_promo': object,'store_promo': object,'dom_promo': object,'dow_promo': object})
beer_test['Date'] = pd.to_datetime(beer_test['Date'])

In [18]:
for c in beer_train.columns:
    col_type = beer_train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        beer_train[c] = beer_train[c].astype('category')

In [19]:
for c in beer_test.columns:
    col_type = beer_test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        beer_test[c] = beer_test[c].astype('category')

In [20]:
print(beer_train['Date'].is_monotonic_increasing)
print(beer_test['Date'].is_monotonic_increasing)
beer_train = beer_train.sort_values(by=['Date'])
beer_test = beer_test.sort_values(by=['Date'])

False
False


In [21]:
# non_numeric_features = ['Store', 'SKU', 'Segment', 'Pack', 'Product', 'Brand','Season',
#        'Birthdays', "New Year's Day", 'New Year Holiday',
#        "The Prophet's Birthday", 'Orthodox Christmas Day', 'Old New Year',
#        "Valentine's Day", 'Defender of the Fatherland Day',
#        'Special Operations Forces Day', "International Women's Day",
#        "Day off for International Women's Day", 'March Equinox',
#        'Orthodox Easter Day', 'Spring and Labor Day', 'Public Holiday',
#        'Victory Day', 'Day off for Victory Day', "Isra and Mi'raj",
#        'Russia Day', 'Ramadan starts', 'June Solstice', 'Lailat al-Qadr',
#        'Eid al-Fitr', 'Day of Knowledge', 'September Equinox', 'Eid al-Adha',
#        'Muharram', 'Unity Day', 'December Solstice',
#        'New Year Holiday Substitution', 'Day off for Spring and Labor Day',
#        'Day off for Russia Day', 'Extra holiday in lieu of Jan. 1',
#        'Extra holiday in lieu of Jan. 7', 'Day off for Unity Day', 'Holiday']

# for feature in non_numeric_features:        
#     beer_train[feature] = LabelEncoder().fit_transform(beer_train[feature])
#     beer_train[feature] = LabelEncoder().fit_transform(beer_train[feature])

In [22]:
X = beer_train.drop(['Sales','Date'], axis=1)#Date
y = beer_train['Sales']

In [23]:
var_categ =[]
for c in X.columns:
    col_type = X[c].dtype
    if col_type.name == 'category':
        var_categ.append(c)

# x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [47]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

# ------------------------------------------------------

# Keras

In [44]:
import keras_tuner as kt
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [45]:
def model_builder(hp):
    '''
    Args:
    hp - Keras tuner object
    '''
    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu', name='dense_1'))
    # Add next layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation='softmax'))
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
    return model

In [48]:
msle = MeanSquaredLogarithmicError()

def model_builder(hp):
    '''
    Args:
    hp - Keras tuner object
    '''
    # Initialize the Sequential API and start stacking the layers
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu', name='dense_1'))
    # Add next layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, activation='softmax'))
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
    return model
def build_model(hp):
    model = tf.keras.Sequential()

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units1, activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
    )

    return model

# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

tuner.search(x_train, y_train, epochs=10, validation_split=0.2)

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_dir\keras_tuner_demo\oracle.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
units1            |96                |?                 
units2            |128               |?                 
units3            |416               |?                 
learning_rate     |0.01              |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |2                 |?                 
tuner/round       |0                 |?                 



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
#     print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

# CATBOOST

In [26]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# import optuna

In [ ]:
# SAMPLE_RATE = 0.4
# RANDOM_SEED = 1
# EARLY_STOPPING_ROUND = 100
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
# X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
# def objective(trial):
#     param = {}
#     param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
#     param['depth'] = trial.suggest_int('depth', 9, 15)
#     param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
#     param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
#     param['grow_policy'] = 'Depthwise'
#     param['iterations'] = 10000
#     param['use_best_model'] = True
#     param['eval_metric'] = 'RMSE'
#     param['od_type'] = 'iter'
#     param['od_wait'] = 20
#     param['random_state'] = RANDOM_SEED
#     param['logging_level'] = 'Silent'
    
#     regressor = CatBoostRegressor(**param)

#     regressor.fit(X_train.copy(), y_train.copy(),
#                   eval_set=[(X_eval.copy(), y_eval.copy())],
#                   early_stopping_rounds=EARLY_STOPPING_ROUND)
#     loss = mean_squared_error(y_valid, regressor.predict(X_valid.copy()))
#     return loss
    

In [ ]:
# %%time
# import warnings
# warnings.filterwarnings('ignore')
# study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
# study.optimize(objective, n_trials=10, n_jobs=-1, timeout=24000)

In [ ]:
# study.best_value

In [ ]:
# study.best_params

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# optuna.visualization.plot_slice(study)

In [ ]:
# optuna.visualization.plot_contour(study, params=['learning_rate',
#                                                  'min_child_samples',
#                                                  'depth',
#                                                  'l2_leaf_reg'])

In [ ]:
# optuna.visualization.plot_param_importances(study)

In [ ]:
# optuna.visualization.plot_edf(study)

In [ ]:
# %%time
# optimized_regressor = CatBoostRegressor(learning_rate=study.best_params['learning_rate'],
#                                         depth=study.best_params['depth'],
#                                         l2_leaf_reg=study.best_params['l2_leaf_reg'],
#                                         min_child_samples=study.best_params['min_child_samples'],
#                                         grow_policy='Depthwise',
#                                         iterations=10000,
#                                         use_best_model=True,
#                                         eval_metric='RMSE',
#                                         od_type='iter',
#                                         od_wait=20,
#                                         random_state=RANDOM_SEED,
#                                         logging_level='Silent')
# optimized_regressor.fit(X_train.copy(), y_train.copy(),
#                         eval_set=[(X_eval.copy(), y_eval.copy())],
#                         early_stopping_rounds=EARLY_STOPPING_ROUND)
# pred_train = optimized_regressor.predict(X_train.copy())
# pred_valid = optimized_regressor.predict(X_valid.copy())

In [27]:
catboost = CatBoostRegressor(
    n_estimators=64,
    learning_rate=0.427912392471475789,
    depth=8,
    bagging_temperature=0.1080748346321565,
    border_count = 210,
    l2_leaf_reg = 10,
    random_strength = 5.03391369357891e-09,
    cat_features=var_categ,
    thread_count=-1,
    silent=True
)

In [29]:
testX = beer_test.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
catboost.fit(X,y)
catboost_prediction = catboost.predict(testX)

beer_test['Sales'] = catboost_prediction
beer_test = beer_test.sort_values(by=['id'])

beer_test[['id','Sales']].to_csv("data/results_catboost.csv", index=False)

# LGBM

In [ ]:
# train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
# %%time
# from lightgbm import LGBMRegressor
# import optuna
# from sklearn.metrics import mean_squared_error
# # from optuna.integration.lightgbm import LGBMRegressor

# def objective(trial):


#     min_child_samples = trial.suggest_int('min_child_samples', 5, 100)
#     num_leaves = trial.suggest_int('num_leaves', 2, 256)
#     learning_rate = trial.suggest_uniform('learning_rate', 0.05, .7)
#     n_estimators = trial.suggest_int('n_estimators', 2, 100)

#     clf = LGBMRegressor(num_leaves=num_leaves,
#                         objective = 'mape',
#                         learning_rate=learning_rate,
#                         n_estimators=n_estimators,
#                         min_child_samples=min_child_samples,
#                         n_jobs=-1
#                        )
    
#     clf.fit(train_x, train_y)
#     y_pred = clf.predict(test_x, num_iteration=clf.best_iteration_)
#     loss = mean_squared_error(test_y, y_pred)
#     return loss

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=10,  n_jobs=-1, timeout=24000)
# print(study.best_trial)

In [ ]:
# study.best_value

In [ ]:
# study.best_params

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# optuna.visualization.plot_slice(study)

In [ ]:
# optuna.visualization.plot_contour(study, params=['learning_rate',
#                                                  'min_child_samples',
#                                                  'num_leaves',
#                                                  'n_estimators'])

In [ ]:
# optuna.visualization.plot_param_importances(study)

In [ ]:
# optuna.visualization.plot_edf(study)

In [ ]:
# %%time
# optimized_regressor = CatBoostRegressor(learning_rate=study.best_params['learning_rate'],
#                                         depth=study.best_params['depth'],
#                                         l2_leaf_reg=study.best_params['l2_leaf_reg'],
#                                         min_child_samples=study.best_params['min_child_samples'],
#                                         grow_policy='Depthwise',
#                                         iterations=10000,
#                                         use_best_model=True,
#                                         eval_metric='RMSE',
#                                         od_type='iter',
#                                         od_wait=20,
#                                         random_state=RANDOM_SEED,
#                                         logging_level='Silent')
# optimized_regressor.fit(X_train.copy(), y_train.copy(),
#                         eval_set=[(X_eval.copy(), y_eval.copy())],
#                         early_stopping_rounds=EARLY_STOPPING_ROUND)
# pred_train = optimized_regressor.predict(X_train.copy())
# pred_valid = optimized_regressor.predict(X_valid.copy())

In [4]:
lgbm = LGBMRegressor(
    boosting_type = 'dart',
    learning_rate = 0.039691509923932582, 
    max_depth = 10,
    metric='rmse',
    n_estimators = 330,
    num_leaves = 21,
    subsample_for_bin = 201000,
    colsample_bytree = 0.8,
    subsample = 1.0,
    silent=True
    )

In [24]:
testX = beer_test.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
lgbm.fit(X,y)
lgbm_prediction = lgbm.predict(testX)

beer_test['Sales'] = lgbm_prediction
beer_test = beer_test.sort_values(by=['id'])

beer_test
beer_test[['id','Sales']].to_csv("data/results_LGBM.csv", index=False)